In [1]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import RandomFlip, RandomRotation, RandomContrast, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1, l2
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import datetime

In [5]:
# Set variables
epoch = 30
batch_size = 32
target_size = (299, 299)

# Load and preprocess images
def load_and_preprocess_images(data_df, image_folder, target_size):
    X = []
    y = []

    for index, row in data_df.iterrows():
        img_filename = row['image_path'][1:]  # Remove leading "/"
        img_path = os.path.join(image_folder, img_filename)
        label = row['label'] - 1

        img = cv2.imread(img_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, target_size)
        img = img / 255.0

        X.append(img)
        y.append(label)

    return np.array(X), np.array(y)

In [3]:
# Load train data
train_df = pd.read_csv("train_images.csv")
script_dir = os.getcwd()
train_images = os.path.join(script_dir, 'train_images')

X_train, y_train = load_and_preprocess_images(train_df, train_images, target_size)

# Shuffle the data
X_train, y_train = shuffle(X_train, y_train, random_state=42)

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [4]:
from sklearn.model_selection import ParameterGrid

# Define a function to create and compile the model with given hyperparameters
def create_model(learning_rate, dropout_rate):
    base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
    avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    dropout = Dropout(dropout_rate)(avg)
    output = tf.keras.layers.Dense(200, activation="softmax")(dropout)
    model = tf.keras.Model(inputs=base_model.input, outputs=output)

    for layer in base_model.layers:
        layer.trainable = False

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

    return model

# Define a grid of hyperparameters to search through
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1],
    'dropout_rate': [0.3, 0.5, 0.7]
}

best_val_accuracy = 0
best_params = {}

# Iterate through each combination of hyperparameters
for params in ParameterGrid(param_grid):
    print(f"Training with params: {params}")
    
    # Create and compile the model with current hyperparameters
    model = create_model(params['learning_rate'], params['dropout_rate'])
    
    # Train the model for one epoch
    history = model.fit(
        X_train,
        y_train,
        epochs=5,
        validation_data=(X_val, y_val)  # Set verbose to 0 to avoid printing training progress
    )
    
    # Evaluate the model on the validation set
    val_accuracy = history.history['val_accuracy'][0]
    print(f"Validation accuracy: {val_accuracy}")
    
    # Check if this set of hyperparameters is the best so far
    if val_accuracy > best_val_accuracy:
        best_val_accuracy = val_accuracy
        best_params = params

print(f"Best validation accuracy: {best_val_accuracy}")
print(f"Best hyperparameters: {best_params}")


Training with params: {'dropout_rate': 0.3, 'learning_rate': 0.001}


Epoch 1/5
99/99 [==============================] - 215s 2s/step - loss: 4.1947 - accuracy: 0.1901 - val_loss: 3.1654 - val_accuracy: 0.3448
Epoch 2/5
99/99 [==============================] - 209s 2s/step - loss: 2.4937 - accuracy: 0.4863 - val_loss: 2.4622 - val_accuracy: 0.4453
Epoch 3/5
99/99 [==============================] - 206s 2s/step - loss: 1.8535 - accuracy: 0.6303 - val_loss: 2.1411 - val_accuracy: 0.4962
Epoch 4/5
99/99 [==============================] - 202s 2s/step - loss: 1.4896 - accuracy: 0.7188 - val_loss: 1.9526 - val_accuracy: 0.5165
Epoch 5/5
99/99 [==============================] - 200s 2s/step - loss: 1.2451 - accuracy: 0.7627 - val_loss: 1.8038 - val_accuracy: 0.5445
Validation accuracy: 0.3447837233543396
Training with params: {'dropout_rate': 0.3, 'learning_rate': 0.01}


Epoch 1/5
99/99 [==============================] - 216s 2s/step - loss: 3.5683 - accuracy: 0.3146 - val_loss: 2.7008 - val_accuracy: 0.4288
Epoch 2/5
99/99 [==============================] - 203s 2s/step - loss: 1.3943 - accuracy: 0.6242 - val_loss: 2.1681 - val_accuracy: 0.5229
Epoch 3/5
99/99 [==============================] - 194s 2s/step - loss: 0.8540 - accuracy: 0.7538 - val_loss: 2.0978 - val_accuracy: 0.5204
Epoch 4/5
99/99 [==============================] - 197s 2s/step - loss: 0.5688 - accuracy: 0.8217 - val_loss: 2.2054 - val_accuracy: 0.5102
Epoch 5/5
99/99 [==============================] - 194s 2s/step - loss: 0.4047 - accuracy: 0.8841 - val_loss: 1.9749 - val_accuracy: 0.5407
Validation accuracy: 0.42875316739082336
Training with params: {'dropout_rate': 0.3, 'learning_rate': 0.1}


Epoch 1/5
99/99 [==============================] - 210s 2s/step - loss: 26.1994 - accuracy: 0.2210 - val_loss: 17.4326 - val_accuracy: 0.3270
Epoch 2/5
99/99 [==============================] - 201s 2s/step - loss: 11.5061 - accuracy: 0.4704 - val_loss: 18.8711 - val_accuracy: 0.3791
Epoch 3/5
99/99 [==============================] - 200s 2s/step - loss: 9.1480 - accuracy: 0.5882 - val_loss: 14.9112 - val_accuracy: 0.4555
Epoch 4/5
99/99 [==============================] - 201s 2s/step - loss: 7.1160 - accuracy: 0.6525 - val_loss: 17.0466 - val_accuracy: 0.4211
Epoch 5/5
99/99 [==============================] - 198s 2s/step - loss: 5.4637 - accuracy: 0.7175 - val_loss: 17.6666 - val_accuracy: 0.4593
Validation accuracy: 0.32697200775146484
Training with params: {'dropout_rate': 0.5, 'learning_rate': 0.001}


Epoch 1/5
99/99 [==============================] - 212s 2s/step - loss: 4.2983 - accuracy: 0.1694 - val_loss: 3.2648 - val_accuracy: 0.3410
Epoch 2/5
99/99 [==============================] - 200s 2s/step - loss: 2.6405 - accuracy: 0.4567 - val_loss: 2.5245 - val_accuracy: 0.4402
Epoch 3/5
99/99 [==============================] - 211s 2s/step - loss: 2.0089 - accuracy: 0.5717 - val_loss: 2.2165 - val_accuracy: 0.4720
Epoch 4/5
99/99 [==============================] - 202s 2s/step - loss: 1.6368 - accuracy: 0.6643 - val_loss: 2.0062 - val_accuracy: 0.5115
Epoch 5/5
99/99 [==============================] - 201s 2s/step - loss: 1.3860 - accuracy: 0.7213 - val_loss: 1.8783 - val_accuracy: 0.5318
Validation accuracy: 0.3409669101238251
Training with params: {'dropout_rate': 0.5, 'learning_rate': 0.01}


Epoch 1/5
99/99 [==============================] - 217s 2s/step - loss: 3.6337 - accuracy: 0.2752 - val_loss: 2.2881 - val_accuracy: 0.4542
Epoch 2/5
99/99 [==============================] - 199s 2s/step - loss: 1.5304 - accuracy: 0.5924 - val_loss: 2.1949 - val_accuracy: 0.4733
Epoch 3/5
99/99 [==============================] - 199s 2s/step - loss: 1.1572 - accuracy: 0.6815 - val_loss: 2.0457 - val_accuracy: 0.5420
Epoch 4/5
99/99 [==============================] - 199s 2s/step - loss: 0.8243 - accuracy: 0.7627 - val_loss: 2.2670 - val_accuracy: 0.4936
Epoch 5/5
99/99 [==============================] - 195s 2s/step - loss: 0.6452 - accuracy: 0.8051 - val_loss: 2.1504 - val_accuracy: 0.5369
Validation accuracy: 0.4541984796524048
Training with params: {'dropout_rate': 0.5, 'learning_rate': 0.1}


Epoch 1/5
99/99 [==============================] - 212s 2s/step - loss: 23.4781 - accuracy: 0.2054 - val_loss: 16.5283 - val_accuracy: 0.3295
Epoch 2/5
99/99 [==============================] - 203s 2s/step - loss: 12.5848 - accuracy: 0.4366 - val_loss: 16.1419 - val_accuracy: 0.3804
Epoch 3/5
99/99 [==============================] - 200s 2s/step - loss: 11.0730 - accuracy: 0.5166 - val_loss: 16.6724 - val_accuracy: 0.4198
Epoch 4/5
99/99 [==============================] - 198s 2s/step - loss: 9.0377 - accuracy: 0.5863 - val_loss: 16.9006 - val_accuracy: 0.4466
Epoch 5/5
99/99 [==============================] - 195s 2s/step - loss: 7.7806 - accuracy: 0.6395 - val_loss: 18.5272 - val_accuracy: 0.4529
Validation accuracy: 0.32951653003692627
Training with params: {'dropout_rate': 0.7, 'learning_rate': 0.001}


Epoch 1/5
99/99 [==============================] - 214s 2s/step - loss: 4.5374 - accuracy: 0.1143 - val_loss: 3.5031 - val_accuracy: 0.3130
Epoch 2/5
99/99 [==============================] - 203s 2s/step - loss: 2.9443 - accuracy: 0.3831 - val_loss: 2.7205 - val_accuracy: 0.3931
Epoch 3/5
99/99 [==============================] - 207s 2s/step - loss: 2.2925 - accuracy: 0.4978 - val_loss: 2.3632 - val_accuracy: 0.4402
Epoch 4/5
99/99 [==============================] - 203s 2s/step - loss: 1.9245 - accuracy: 0.5646 - val_loss: 2.1412 - val_accuracy: 0.4898
Epoch 5/5
99/99 [==============================] - 199s 2s/step - loss: 1.6627 - accuracy: 0.6354 - val_loss: 1.9979 - val_accuracy: 0.5153
Validation accuracy: 0.3129771053791046
Training with params: {'dropout_rate': 0.7, 'learning_rate': 0.01}


Epoch 1/5
99/99 [==============================] - 213s 2s/step - loss: 4.1419 - accuracy: 0.2172 - val_loss: 2.4383 - val_accuracy: 0.4008
Epoch 2/5
99/99 [==============================] - 201s 2s/step - loss: 2.4438 - accuracy: 0.4395 - val_loss: 2.3647 - val_accuracy: 0.4618
Epoch 3/5
99/99 [==============================] - 199s 2s/step - loss: 2.0495 - accuracy: 0.5261 - val_loss: 2.1684 - val_accuracy: 0.5051
Epoch 4/5
99/99 [==============================] - 200s 2s/step - loss: 1.7080 - accuracy: 0.5873 - val_loss: 2.3114 - val_accuracy: 0.4771
Epoch 5/5
99/99 [==============================] - 200s 2s/step - loss: 1.5825 - accuracy: 0.6210 - val_loss: 2.2415 - val_accuracy: 0.5331
Validation accuracy: 0.4007633626461029
Training with params: {'dropout_rate': 0.7, 'learning_rate': 0.1}


Epoch 1/5
99/99 [==============================] - 213s 2s/step - loss: 19.4139 - accuracy: 0.1545 - val_loss: 15.9813 - val_accuracy: 0.3193
Epoch 2/5
99/99 [==============================] - 204s 2s/step - loss: 17.8582 - accuracy: 0.3315 - val_loss: 15.8487 - val_accuracy: 0.3766
Epoch 3/5
99/99 [==============================] - 197s 2s/step - loss: 16.5330 - accuracy: 0.4194 - val_loss: 14.8505 - val_accuracy: 0.4631
Epoch 4/5
99/99 [==============================] - 198s 2s/step - loss: 14.5345 - accuracy: 0.4777 - val_loss: 15.2455 - val_accuracy: 0.4567
Epoch 5/5
99/99 [==============================] - 198s 2s/step - loss: 14.4868 - accuracy: 0.5096 - val_loss: 18.1248 - val_accuracy: 0.4288
Validation accuracy: 0.3193384110927582
Best validation accuracy: 0.4541984796524048
Best hyperparameters: {'dropout_rate': 0.5, 'learning_rate': 0.01}


In [6]:
# Model architecture
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras import regularizers
base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
dropout = Dropout(0.3)(avg) 
output = tf.keras.layers.Dense(200, activation="softmax")(dropout)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

#output = tf.keras.layers.Dense(200, activation="softmax")(dropout)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False
    
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [5]:
# Train the model with early stopping
model.fit(
    X_train,
    y_train,
    epochs=epoch,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping] 
)

Epoch 1/30
99/99 [==============================] - 213s 2s/step - loss: 4.1784 - accuracy: 0.1924 - val_loss: 3.1652 - val_accuracy: 0.3193
Epoch 2/30
99/99 [==============================] - 204s 2s/step - loss: 2.5016 - accuracy: 0.4838 - val_loss: 2.4705 - val_accuracy: 0.4338
Epoch 3/30
99/99 [==============================] - 202s 2s/step - loss: 1.8580 - accuracy: 0.6242 - val_loss: 2.1302 - val_accuracy: 0.4847
Epoch 4/30
99/99 [==============================] - 201s 2s/step - loss: 1.4834 - accuracy: 0.7108 - val_loss: 1.9386 - val_accuracy: 0.5305
Epoch 5/30
99/99 [==============================] - 199s 2s/step - loss: 1.2434 - accuracy: 0.7650 - val_loss: 1.8184 - val_accuracy: 0.5382
Epoch 6/30
99/99 [==============================] - 200s 2s/step - loss: 1.0642 - accuracy: 0.8080 - val_loss: 1.7202 - val_accuracy: 0.5623
Epoch 7/30
99/99 [==============================] - 201s 2s/step - loss: 0.9349 - accuracy: 0.8303 - val_loss: 1.6465 - val_accuracy: 0.5852
Epoch 8/30
99

In [6]:
# Load test data paths
test_df = pd.read_csv("test_images_path.csv")
test_images = os.path.join(script_dir, 'test_images')
X_test, _ = load_and_preprocess_images(test_df, test_images, target_size)

In [7]:
# Make predictions
predictions = model.predict(X_test)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1) + 1

# Update the 'label' column in the test dataframe with the predicted labels
test_df['label'] = predicted_labels

selected_columns = ['id', 'label']
test_df_selected = test_df[selected_columns]

# Generate a timestamp for the filename
current_datetime = datetime.datetime.now()
timestamp = current_datetime.strftime("%Y%m%d_%H%M")
file_name = f"submission_{timestamp}.csv"

# Define the folder for submissions
submissions_folder = os.path.join(script_dir, 'submissions')

# Specify the full file path
file_path = os.path.join(submissions_folder, file_name)

# Save the final submission file
test_df_selected.to_csv(file_path, index=False)

125/125 [==============================] - 213s 2s/step


In [ ]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.layers import Flatten, Dense, BatchNormalization
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from sklearn.model_selection import GridSearchCV

# Define a function to create the model
def create_model(learning_rate=0.001):
    base_model = InceptionV3(weights='imagenet', include_top=False, input_shape=(299, 299, 3))
    for layer in base_model.layers:
        layer.trainable = False

    base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
    avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    dropout = tf.keras.layers.Dropout(0.5)(avg)
    output = tf.keras.layers.Dense(200, activation="softmax")(dropout)
    model = tf.keras.Model(inputs=base_model.input, outputs=output)

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Define hyperparameters to search
param_grid = {
    'learning_rate': [0.001, 0.01]
}

# Create a KerasClassifier based on the defined model
keras_model = create_model()
keras_classifier = GridSearchCV(estimator=keras_model, param_grid=param_grid, cv=3)

# Train the model with GridSearchCV
keras_classifier.fit(X_train, y_train)

# Get the best parameters and best score
best_params = keras_classifier.best_params_
best_score = keras_classifier.best_score_

print("Best parameters:", best_params)
print("Best score:", best_score)


In [9]:
# Define hyperparameters
dropout_rates = [0.3, 0.5, 0.7]
learning_rates = [0.001, 0.01, 0.1]

best_score = 0
best_params = {}


import tensorflow as tf
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

# Function to create the model
def create_model(dropout_rate=0.5, learning_rate=0.001):
    base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
    avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    dropout = tf.keras.layers.Dropout(dropout_rate)(avg)
    output = tf.keras.layers.Dense(200, activation="softmax")(dropout)
    model = tf.keras.Model(inputs=base_model.input, outputs=output)

    for layer in base_model.layers:
        layer.trainable = False

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
    
    return model


# Loop through hyperparameters and train models
for dropout_rate in dropout_rates:
    for learning_rate in learning_rates:
        # Initialize scores for current hyperparameter combination
        scores = []
        
        # Perform K-Fold cross-validation
        kf = KFold(n_splits=3)
        for train_index, val_index in kf.split(X_train):
            X_cv_train, X_cv_val = X_train[train_index], X_train[val_index]
            y_cv_train, y_cv_val = y_train[train_index], y_train[val_index]
            
            model = create_model(dropout_rate=dropout_rate, learning_rate=learning_rate)
            
            # Train the model
            model.fit(X_cv_train, y_cv_train, epochs=epoch, batch_size=batch_size, verbose=0)
            
            # Evaluate on validation data
            _, acc = model.evaluate(X_cv_val, y_cv_val, verbose=0)
            scores.append(acc)
        
        # Calculate average score for the current hyperparameter combination
        avg_score = np.mean(scores)
        
        # Check if current configuration is the best
        if avg_score > best_score:
            best_score = avg_score
            best_params['dropout_rate'] = dropout_rate
            best_params['learning_rate'] = learning_rate

# Output best parameters and score
print("Best Score:", best_score)
print("Best Parameters:", best_params)

KeyboardInterrupt: 

In [5]:
# Model architecture
base_model = tf.keras.applications.xception.Xception(weights="imagenet", include_top=False)
avg = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
dropout = Dropout(0.5)(avg) 
output = tf.keras.layers.Dense(200, activation="softmax")(dropout)
model = tf.keras.Model(inputs=base_model.input, outputs=output)

for layer in base_model.layers:
    layer.trainable = False
    
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])

# Implement early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

2023-12-08 15:46:18.927934: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-08 15:46:19.441676: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 18209 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB MIG 3g.20gb, pci bus id: 0000:32:00.0, compute capability: 8.0


In [6]:
# Train the model with early stopping
model.fit(
    X_train,
    y_train,
    epochs=epoch,
    validation_data=(X_val, y_val),
    callbacks=[early_stopping] 
)

Epoch 1/20


2023-12-08 15:46:34.260783: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8401
2023-12-08 15:46:37.512234: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-12-08 15:46:37.532664: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x265129b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-12-08 15:46:37.532695: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB MIG 3g.20gb, Compute Capability 8.0
2023-12-08 15:46:37.560280: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.


 1/99 [..............................] - ETA: 13:53 - loss: 5.4000 - accuracy: 0.0000e+00

2023-12-08 15:46:37.849943: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


99/99 [==============================] - 20s 116ms/step - loss: 4.3191 - accuracy: 0.1691 - val_loss: 3.2647 - val_accuracy: 0.3257
Epoch 2/20
99/99 [==============================] - 8s 80ms/step - loss: 2.6462 - accuracy: 0.4522 - val_loss: 2.5450 - val_accuracy: 0.4300
Epoch 3/20
99/99 [==============================] - 8s 81ms/step - loss: 2.0050 - accuracy: 0.5799 - val_loss: 2.2115 - val_accuracy: 0.4809
Epoch 4/20
99/99 [==============================] - 8s 80ms/step - loss: 1.6280 - accuracy: 0.6669 - val_loss: 2.0168 - val_accuracy: 0.5089
Epoch 5/20
99/99 [==============================] - 8s 81ms/step - loss: 1.3804 - accuracy: 0.7290 - val_loss: 1.8813 - val_accuracy: 0.5331
Epoch 6/20
99/99 [==============================] - 8s 80ms/step - loss: 1.2020 - accuracy: 0.7634 - val_loss: 1.7887 - val_accuracy: 0.5496
Epoch 7/20
99/99 [==============================] - 8s 81ms/step - loss: 1.0612 - accuracy: 0.7959 - val_loss: 1.7123 - val_accuracy: 0.5573
Epoch 8/20
99/99 [====

In [7]:
# Load test data paths
test_df = pd.read_csv("test_images_path.csv")
test_images = os.path.join(script_dir, 'test_images')
X_test, _ = load_and_preprocess_images(test_df, test_images, target_size)

In [8]:
# Make predictions
predictions = model.predict(X_test)

# Convert predictions to class labels
predicted_labels = np.argmax(predictions, axis=1) + 1

# Update the 'label' column in the test dataframe with the predicted labels
test_df['label'] = predicted_labels

selected_columns = ['id', 'label']
test_df_selected = test_df[selected_columns]

# Generate a timestamp for the filename
current_datetime = datetime.datetime.now()
timestamp = current_datetime.strftime("%Y%m%d_%H%M")
file_name = f"submission_{timestamp}.csv"

# Define the folder for submissions
submissions_folder = os.path.join(script_dir, 'submissions')

# Specify the full file path
file_path = os.path.join(submissions_folder, file_name)

# Save the final submission file
test_df_selected.to_csv(file_path, index=False)


125/125 [==============================] - 9s 62ms/step
